In [1]:
import tensorflow as tf
import numpy as np

2024-04-15 09:38:26.405152: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
class Linear(tf.keras.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=[input_shape[-1],self.units], initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=[self.units,], initializer='random_normal',trainable=True)
        super(Linear,self).build(input_shape)
    @tf.function
    def call(self, inputs):
        return tf.matmul(inputs,self.w) + self.b
    
    def get_config(self):
        config = super(Linear, self).get_config()
        config.update({"units": self.units})
        return config
linear = Linear(units=9)
print(linear.built)
linear.build(input_shape=(None,10))
print(linear.built)

False
True


In [8]:
linear = Linear(units = 8)
print(linear.built)
linear.build(input_shape = (None,16)) 
print(linear.compute_output_shape(input_shape = (None,16)))

False


NotImplementedError: Layer Linear should implement `def compute_output_shape(self, input_shape)`.

In [15]:
linear = Linear(units = 16)
print(linear.built)
#如果built = False，调用__call__时会先调用build方法, 再调用call方法。
y = linear(tf.random.uniform((100,64))) 
print(linear.built)
config = linear.get_config()
print(config)

False
True
{'name': 'linear_11', 'trainable': True, 'dtype': 'float32', 'units': 16}


In [16]:
y

<tf.Tensor: shape=(100, 16), dtype=float32, numpy=
array([[-0.13274516,  0.12991844,  0.0550682 , ..., -0.08484762,
        -0.27725664,  0.34248954],
       [-0.19785546,  0.13795723,  0.09373994, ..., -0.14870182,
        -0.43288058,  0.2735843 ],
       [-0.2568862 ,  0.2142141 , -0.07396038, ...,  0.01163568,
        -0.45295444,  0.18708257],
       ...,
       [-0.02963001,  0.12725076, -0.04176406, ...,  0.14848326,
        -0.5030447 ,  0.07789537],
       [-0.34988523,  0.14317064,  0.11104941, ...,  0.10179436,
        -0.5543718 ,  0.51664907],
       [-0.4557419 ,  0.17383856, -0.15458207, ...,  0.12212983,
        -0.39436463,  0.2511169 ]], dtype=float32)>

In [20]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential()
#注意该处的input_shape会被模型加工，无需使用None代表样本数量维
model.add(Linear(units = 1,input_shape = (2,)))  
model.summary()
model.compile(optimizer = "sgd",loss = "mse",metrics=["mae"])
print(model.predict(tf.constant([[3.0,2.0],[4.0,5.0]])))

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ linear (Linear)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 797ms/step
[[0.23502578]
 [0.52773756]]
